https://aws-data-wrangler.readthedocs.io/en/stable/stubs/awswrangler.s3.read_csv.html

In [1]:
pip install awswrangler

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install haversine

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime

import json
from haversine import haversine, Unit
from scipy.spatial import distance

In [4]:
input_bucket = 'adsb-data'
input_subfolder = ''

output_bucket = 'partly-cloudy-common-area'
output_subfolder = ''

In [5]:
from sagemaker import get_execution_role
role = get_execution_role()

In [6]:
# Examine the files in the 'adsb-data s3 Bucket
conn = boto3.client('s3')
contents = conn.list_objects(Bucket= input_bucket, Prefix= input_subfolder)['Contents']
for f in contents:
    print(f['Key'])

2016_06_20_0000Z.json
2016_06_20_0001Z.json
2016_06_20_0002Z.json
2016_06_20_0003Z.json
2016_06_20_0004Z.json
2016_06_20_0005Z.json
2016_06_20_0006Z.json
2016_06_20_0007Z.json
2016_06_20_0008Z.json
2016_06_20_0009Z.json
2016_06_20_0010Z.json
2016_06_20_0011Z.json
2016_06_20_0012Z.json
2016_06_20_0013Z.json
2016_06_20_0014Z.json
2016_06_20_0015Z.json
2016_06_20_0016Z.json
2016_06_20_0017Z.json
2016_06_20_0018Z.json
2016_06_20_0019Z.json
2016_06_20_0020Z.json
2016_06_20_0021Z.json
2016_06_20_0022Z.json
2016_06_20_0023Z.json
2016_06_20_0024Z.json
2016_06_20_0025Z.json
2016_06_20_0026Z.json
2016_06_20_0027Z.json
2016_06_20_0028Z.json
2016_06_20_0029Z.json
2016_06_20_0030Z.json
2016_06_20_0031Z.json
2016_06_20_0032Z.json
2016_06_20_0033Z.json
2016_06_20_0034Z.json
2016_06_20_0035Z.json
2016_06_20_0036Z.json
2016_06_20_0037Z.json
2016_06_20_0038Z.json
2016_06_20_0039Z.json
2016_06_20_0040Z.json
2016_06_20_0041Z.json
2016_06_20_0042Z.json
2016_06_20_0043Z.json
2016_06_20_0044Z.json
2016_06_20

In [7]:
# Examine the contents in the 'adsb-data' s3 Bucket
contents

[{'Key': '2016_06_20_0000Z.json',
  'LastModified': datetime.datetime(2021, 6, 18, 23, 23, 7, tzinfo=tzlocal()),
  'ETag': '"d56d65c15bf41fd73b773129955981b8"',
  'Size': 5671687,
  'StorageClass': 'STANDARD'},
 {'Key': '2016_06_20_0001Z.json',
  'LastModified': datetime.datetime(2021, 6, 18, 23, 23, 7, tzinfo=tzlocal()),
  'ETag': '"2c21bd5d89918c4d18aad08a40c60a79"',
  'Size': 5808611,
  'StorageClass': 'STANDARD'},
 {'Key': '2016_06_20_0002Z.json',
  'LastModified': datetime.datetime(2021, 6, 18, 23, 23, 8, tzinfo=tzlocal()),
  'ETag': '"68a0c68a416fb7cbbf5b757fe08fcdb9"',
  'Size': 5751906,
  'StorageClass': 'STANDARD'},
 {'Key': '2016_06_20_0003Z.json',
  'LastModified': datetime.datetime(2021, 6, 18, 23, 23, 9, tzinfo=tzlocal()),
  'ETag': '"09c7cbd75f26c7783a375a0064e9a5f7"',
  'Size': 5737587,
  'StorageClass': 'STANDARD'},
 {'Key': '2016_06_20_0004Z.json',
  'LastModified': datetime.datetime(2021, 6, 18, 23, 23, 10, tzinfo=tzlocal()),
  'ETag': '"25c22041a9ca3f84669642d89ed9a6

In [8]:
# Get all file names in 'adsb-data' Bucket into a list 
myFileNames = [f['Key'] for f in contents] # <---------------------- Using 'AWS contents' does not work!!!
# myFileNames = ['2016_06_20_0002Z.json', '2016_06_20_0003Z.json']

In [9]:
# Generate all file names assume to be in 'adsb-data' <--------------------------------- Redo by Iteration
dateTimes = pd.date_range('2016-06-20 00:00:00', '2016-06-20 23:59:00', freq= 'T')
dateTimesSer = pd.Series([str(dateTime) for dateTime in dateTimes], name= 'temp')
myFileNames = [dateTimesSer[i][0:4] + '_' + dateTimesSer[i][5:7] + '_' + dateTimesSer[i][8:10] + '_' + dateTimesSer[i][11:13] + dateTimesSer[i][14:16] +'Z.json' for i in range(len(dateTimesSer))]

In [10]:
myFileNames

['2016_06_20_0000Z.json',
 '2016_06_20_0001Z.json',
 '2016_06_20_0002Z.json',
 '2016_06_20_0003Z.json',
 '2016_06_20_0004Z.json',
 '2016_06_20_0005Z.json',
 '2016_06_20_0006Z.json',
 '2016_06_20_0007Z.json',
 '2016_06_20_0008Z.json',
 '2016_06_20_0009Z.json',
 '2016_06_20_0010Z.json',
 '2016_06_20_0011Z.json',
 '2016_06_20_0012Z.json',
 '2016_06_20_0013Z.json',
 '2016_06_20_0014Z.json',
 '2016_06_20_0015Z.json',
 '2016_06_20_0016Z.json',
 '2016_06_20_0017Z.json',
 '2016_06_20_0018Z.json',
 '2016_06_20_0019Z.json',
 '2016_06_20_0020Z.json',
 '2016_06_20_0021Z.json',
 '2016_06_20_0022Z.json',
 '2016_06_20_0023Z.json',
 '2016_06_20_0024Z.json',
 '2016_06_20_0025Z.json',
 '2016_06_20_0026Z.json',
 '2016_06_20_0027Z.json',
 '2016_06_20_0028Z.json',
 '2016_06_20_0029Z.json',
 '2016_06_20_0030Z.json',
 '2016_06_20_0031Z.json',
 '2016_06_20_0032Z.json',
 '2016_06_20_0033Z.json',
 '2016_06_20_0034Z.json',
 '2016_06_20_0035Z.json',
 '2016_06_20_0036Z.json',
 '2016_06_20_0037Z.json',
 '2016_06_20

In [11]:
%%time
# Obtain necessary data items from all json files in 'adsb-data' in a single dataframe

dt = []
eyedee = []
alt = []
lat = []
long = []

for myFileName in myFileNames:
    s3_obj = boto3.client('s3')
    s3_myobj = s3_obj.get_object(Bucket= input_bucket, Key= myFileName)
    s3_mydata = s3_myobj['Body'].read().decode('utf-8')
    js = json.loads(s3_mydata)
    
    dattim = datetime(year= int(myFileName[0:4]),
                      month= int(myFileName[5:7]),
                      day= int(myFileName[8:10]),
                      hour= int(myFileName[11:13]),
                      minute= int(myFileName[13:15]),
                      second= 0)
    
    for i in range(len(js['acList'])):
        dt.append(str(dattim))
        if "Id" in js['acList'][i].keys():
            eyedee.append(js['acList'][i]['Id'])
        else:
            eyedee.append(np.nan)
        if "Alt" in js['acList'][i].keys():
            alt.append(js['acList'][i]['Alt'])
        else:
            alt.append(np.nan)
        if ("Lat" in js['acList'][i].keys()):
            if type(js['acList'][i]['Lat']) == np.float:
                lat.append(js['acList'][i]['Lat'])
            else:
                lat.append(np.nan)
        else:
            lat.append(np.nan)
        if "Long" in js['acList'][i].keys():
            long.append(js['acList'][i]['Long'])
        else:
            long.append(np.nan)


CPU times: user 5min 49s, sys: 10.3 s, total: 5min 59s
Wall time: 10min 35s


In [12]:
df = pd.DataFrame({"DateTime":dt, "id":eyedee, 'Altitude':alt, 'Latitude':lat, 'Longitude':long})

In [32]:
df

,DateTime,id,Altitude,Latitude,Longitude
0,2016-06-20 00:00:00,8721588,31000.0,NaN,NaN
1,2016-06-20 00:00:00,2961715,NaN,0.000000,-2147.483648
2,2016-06-20 00:00:00,10607858,700.0,NaN,NaN
3,2016-06-20 00:00:00,12608926,2775.0,NaN,NaN
4,2016-06-20 00:00:00,7786027,300.0,1.334150,103.987221
...,...,...,...,...,...
6743149,2016-06-20 23:59:00,11401618,6950.0,NaN,NaN
6743150,2016-06-20 23:59:00,11401617,3407.0,NaN,NaN
6743151,2016-06-20 23:59:00,5271422,1000.0,45.481979,13.648616
6743152,2016-06-20 23:59:00,11401619,6752.0,NaN,NaN


In [33]:
# Examine number of records with NULL
df.isnull().sum()

DateTime          0
id                0
Altitude      90618
Latitude     718818
Longitude    718818
dtype: int64

In [34]:
df['DateTime'].value_counts()

2016-06-20 15:43:00    6711
2016-06-20 15:38:00    6694
2016-06-20 14:52:00    6656
2016-06-20 14:44:00    6649
2016-06-20 14:55:00    6648
                       ... 
2016-06-20 04:13:00    2668
2016-06-20 04:06:00    2667
2016-06-20 04:10:00    2643
2016-06-20 04:09:00    2629
2016-06-20 03:07:00     736
Name: DateTime, Length: 1439, dtype: int64

In [35]:
df_select = df.dropna(axis= 0, how= 'any').copy() # Per Dr. Cherry on 6/17/2021

In [36]:
df_select

,DateTime,id,Altitude,Latitude,Longitude
4,2016-06-20 00:00:00,7786027,300.0,1.334150,103.987221
13,2016-06-20 00:00:00,2900289,3752492.0,0.000003,0.000000
23,2016-06-20 00:00:00,4196356,2900.0,51.487190,-0.581210
24,2016-06-20 00:00:00,4494056,25450.0,51.418110,2.221830
25,2016-06-20 00:00:00,11287741,39525.0,50.827541,0.748670
...,...,...,...,...,...
6743117,2016-06-20 23:59:00,8189898,0.0,-33.870198,151.210213
6743128,2016-06-20 23:59:00,12193915,55800.0,37.812332,-120.652634
6743130,2016-06-20 23:59:00,3289137,3159599.0,0.000706,0.000000
6743133,2016-06-20 23:59:00,2894892,2894892.0,0.000000,-2147.483648


In [37]:
latlong_bucket = 'partly-cloudy-common-area'
latlongFileName = 'latlonRAP.csv'
latlongRAP = wr.s3.read_csv(path= f"s3://{latlong_bucket}/{latlongFileName}")

In [38]:
latlongRAP

,Nx,Ny,Lat,Lon
0,1,1,16.281000,-126.138000
1,2,1,16.322011,-125.954684
2,3,1,16.362789,-125.771252
3,4,1,16.403332,-125.587705
4,5,1,16.443642,-125.404045
...,...,...,...,...
67720,297,225,55.648911,-58.431595
67721,298,225,55.607604,-58.167947
67722,299,225,55.565986,-57.904583
67723,300,225,55.524058,-57.641507


In [39]:
minLatRAP = min(latlongRAP['Lat']) - 0.04
maxLatRAP = max(latlongRAP['Lat']) + 0.04
minLonRAP = min(latlongRAP['Lon']) - 0.2
maxLonRAP = max(latlongRAP['Lon']) + 0.2

In [40]:
adsb_conus = df_select.loc[(df_select['Latitude'] >= minLatRAP) & 
                          (df_select['Latitude'] <= maxLatRAP) &
                          (df_select['Longitude'] >= minLonRAP) & 
                          (df_select['Longitude'] <= maxLonRAP)].copy()

In [41]:
adsb_conus.reset_index(inplace= True)

In [42]:
adsb_conus

,index,DateTime,id,Altitude,Latitude,Longitude
0,29,2016-06-20 00:00:00,4196601,2350.0,36.076401,-115.127663
1,31,2016-06-20 00:00:00,11069170,10675.0,35.138141,-80.789509
2,36,2016-06-20 00:00:00,10700817,7400.0,30.632210,-81.586450
3,52,2016-06-20 00:00:00,11281093,1575.0,40.742472,-73.840173
4,55,2016-06-20 00:00:00,12598340,37000.0,45.256130,-81.588492
...,...,...,...,...,...,...
3153946,6743065,2016-06-20 23:59:00,11226973,75.0,42.375396,-71.005932
3153947,6743066,2016-06-20 23:59:00,11165706,33975.0,39.073242,-82.903549
3153948,6743069,2016-06-20 23:59:00,11364484,13075.0,33.685525,-111.892433
3153949,6743107,2016-06-20 23:59:00,11402207,12150.0,41.788536,-87.750229


In [43]:
# Obtain NOAA RAP hPa (to the nearest 25 hPa) from altitude value in feet
def alt_ft_to_nearest_hPa(alt):
    if alt > 51805: # upper limit altitude of 100 hPa (51805 ft)
        return 100
    elif alt < 364: # lower limit altitude of 1000 hPa (364 ft)
        return 1000
    else:
        a = (1/0.190284)
        b =  alt/145366.45
        P_hpa  = ((1-b)**a) * 1013.25
        return int(25 * round(P_hpa/25))

In [44]:
ls = list(adsb_conus['Altitude'])
nearest_hpa = [alt_ft_to_nearest_hPa(i) for i  in ls]

adsb_conus['hPa'] = nearest_hpa

In [45]:
adsb_conus

,index,DateTime,id,Altitude,Latitude,Longitude,hPa
0,29,2016-06-20 00:00:00,4196601,2350.0,36.076401,-115.127663,925
1,31,2016-06-20 00:00:00,11069170,10675.0,35.138141,-80.789509,675
2,36,2016-06-20 00:00:00,10700817,7400.0,30.632210,-81.586450,775
3,52,2016-06-20 00:00:00,11281093,1575.0,40.742472,-73.840173,950
4,55,2016-06-20 00:00:00,12598340,37000.0,45.256130,-81.588492,225
...,...,...,...,...,...,...,...
3153946,6743065,2016-06-20 23:59:00,11226973,75.0,42.375396,-71.005932,1000
3153947,6743066,2016-06-20 23:59:00,11165706,33975.0,39.073242,-82.903549,250
3153948,6743069,2016-06-20 23:59:00,11364484,13075.0,33.685525,-111.892433,625
3153949,6743107,2016-06-20 23:59:00,11402207,12150.0,41.788536,-87.750229,650


In [46]:
# Develop lists of lat,long tuples for haversine
adsb_coord = list(zip(adsb_conus['Latitude'], adsb_conus['Longitude'])) # a list of (Latitude, Longitude) tuples from "adsb_conus"
latlonRAP_coord = list(zip(latlongRAP['Lat'], latlongRAP['Lon'])) # a list of (Lat, Lon) tuples from "latlongRAP"

In [51]:
%%time
from datetime import datetime

# for each (Latitude, Longitude) tuple in "adsb_coord" list, find the index in 'latlonRAP_coord' that has the minimum distance to (Lat, Lon) tuple 
idx = []
for i in range(len(adsb_coord)):
    idx.append(np.argmin([haversine(adsb_coord[i], latlonRAP_coord[j]) for j in range(len(latlonRAP_coord))]))
    print(i, datetime.now().time())

0 11:26:11.718155
1 11:26:11.866882
2 11:26:12.015161
3 11:26:12.166289
4 11:26:12.315606
5 11:26:12.464073
6 11:26:12.612832
7 11:26:12.761076
8 11:26:12.910236
9 11:26:13.060758
10 11:26:13.210210
11 11:26:13.358859
12 11:26:13.510449
13 11:26:13.659493
14 11:26:13.809629
15 11:26:13.957508
16 11:26:14.105917
17 11:26:14.253988
18 11:26:14.402767
19 11:26:14.551778
20 11:26:14.700898
21 11:26:14.849753
22 11:26:14.998310
23 11:26:15.145602
24 11:26:15.293282
25 11:26:15.442673
26 11:26:15.592056
27 11:26:15.738887
28 11:26:15.887669
29 11:26:16.036538
30 11:26:16.185813
31 11:26:16.335429
32 11:26:16.485186
33 11:26:16.633642
34 11:26:16.782794
35 11:26:16.931485
36 11:26:17.080344
37 11:26:17.227354
38 11:26:17.375593
39 11:26:17.523618
40 11:26:17.672890
41 11:26:17.821087
42 11:26:17.969629
43 11:26:18.116764
44 11:26:18.266742
45 11:26:18.414911
46 11:26:18.564782
47 11:26:18.712289
48 11:26:18.860623
49 11:26:19.007791
50 11:26:19.156613
51 11:26:19.304511
52 11:26:19.453738
53 

KeyboardInterrupt: 

In [ ]:
x = np.array(latlongRAP['Nx']) # All "Nx" in 'latlongRAP' in an array
y = np.array(latlongRAP['Ny']) # All "Ny" in 'latlongRAP' in an array

adsb_conus['Nx'] = list(x[idx]) # "Nx" values with indexes with minimum distance
adsb_conus['Ny'] = list(y[idx]) # "Ny" values with indexes with minimum distance

In [13]:
# Examine the list of files are already in the output_bucket (i.e., "partly-cloudy-common-area")
conn = boto3.client('s3')
contents = conn.list_objects(Bucket= output_bucket, Prefix= output_subfolder)['Contents']
for f in contents:
    print(f['Key'])

JuneFirst2020_24hr_issr.csv
hourly_issr_summary.csv


In [14]:
outputFileName = 'adsb_conus.parquet' # <--------------------------------------------- Specify

wr.s3.to_csv(df_select, f"s3://{output_bucket}/{outputFileName}", index=False)

In [ ]:
wr.s3.to_parquet(adsb_conus, f"s3://{output_bucket}/{outputFileName}")